In [1]:
import gym
import numpy as np

In [36]:
env = gym.make('gym_gridworld:gridworld-v4', size=3, player_pos=[0,0], goal_pos=[[2,2]]) 

In [3]:
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

In [4]:
def tabularTDZero(policy=[0.25,0.25,0.25,0.25], alpha=0.1, gamma=1):
    V = [0 for i in env.observation_space]
    
    for i in range(100000):
        obs0 = env.reset()
        
        done = False
        while not done:
            action = np.random.choice([UP, RIGHT, DOWN, LEFT], p=policy)
            obs1, reward, done, info = env.step(action)
            
            obs0_i = env.observation_space.index(obs0)
            obs1_i = env.observation_space.index(obs1)
            
            V[obs0_i] = V[obs0_i] + alpha * (reward + gamma * V[obs1_i] - V[obs0_i])
            
            obs0 = obs1
        
    return V

In [5]:
a = tabularTDZero()

In [6]:
np.array(a).reshape(3,3)

array([[-26.63526579, -24.01310281, -22.12160586],
       [-24.71451945, -21.64036123, -15.52603753],
       [-22.73708552, -16.42409977,   0.        ]])

In [7]:
env = gym.make('gym_gridworld:gridworld-v4', size=4, player_pos=[0,3], goal_pos=[[3,3], [0,0]]) 

In [8]:
a = tabularTDZero()
np.array(a).reshape(4,4)

array([[  0.        ,  -8.51834845, -17.51567378, -19.7675834 ],
       [-13.35316172, -15.54478301, -17.99881873, -16.48236428],
       [-17.36659171, -18.55989591, -16.90376001,  -9.96660682],
       [-20.70701166, -19.89462409, -14.49348776,   0.        ]])

In [64]:
def sarsa(alpha=0.1, epsilon=0.1, gamma=1):
    Q = {a: [0 for s in env.observation_space] for a in [UP, RIGHT, DOWN, LEFT]}
    
    for i in range(10000):
        obs0 = env.reset()
        obs0_i = env.observation_space.index(obs0)
        action0 = max({a: Q[a][obs0_i] for a in Q}, key=Q.get)
        action0 = np.random.choice([action0, UP, RIGHT, DOWN, LEFT], p=[1-epsilon,epsilon/4,epsilon/4,epsilon/4,epsilon/4])
        
        done = False
        while not done:
            obs1, reward, done, info = env.step(action0)
            
            obs1_i = env.observation_space.index(obs1)
            action1 = max({a: Q[a][obs1_i] for a in Q}, key=Q.get)
            action1 = np.random.choice([action1, UP, RIGHT, DOWN, LEFT], p=[1-epsilon,epsilon/4,epsilon/4,epsilon/4,epsilon/4])
            
            Q[action0][obs0_i] = Q[action0][obs0_i] + alpha * (reward + gamma * Q[action1][obs1_i] - Q[action0][obs0_i])
            
            obs0_i = obs1_i
            action0 = action1
            
        
    return Q

In [65]:
a_sarsa = sarsa()

In [66]:
obs0 = env.reset()

In [79]:
obs0_i = env.observation_space.index(obs0)
action = max({a: a_sarsa[a][obs0_i] for a in a_sarsa}, key=a_sarsa.get)

In [80]:
env.render()

[[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [81]:
obs0, reward, done, info = env.step(action)